<a href="https://colab.research.google.com/github/Rosenzweigova/dw_matrix_road_sign/blob/master/M3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install hyperopt

In [21]:
import pandas as pd
import numpy as np

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

%load_ext tensorboard

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
# Ładujemy dane
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1:y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
# Trenowanie modelu
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])


  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  # Normalizujemy macierz y_test
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v6(input_shape, num_classes):
    return Sequential([
        # Podwajamy liczbę filtrów                                      
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3), # powoduje że model się nie przyucza

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(0.3),    

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(0.3),            

        Flatten(),

        Dense(1024, activation = 'relu'), 
        Dropout(0.3),

        Dense(1024, activation = 'relu'), 
        Dropout(0.3),

        Dense(num_classes, activation = 'softmax'),         
    ])

In [15]:
# Uruchomienie

model = get_cnn_v6(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.5020 - accuracy: 0.3058 - val_loss: 0.9543 - val_accuracy: 0.7109
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.8422 - accuracy: 0.7235 - val_loss: 0.2747 - val_accuracy: 0.9065
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4054 - accuracy: 0.8709 - val_loss: 0.1225 - val_accuracy: 0.9614
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2585 - accuracy: 0.9197 - val_loss: 0.0731 - val_accuracy: 0.9777
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2006 - accuracy: 0.9397 - val_loss: 0.0439 - val_accuracy: 0.9873


0.9587301587301588

In [17]:
model_trained.evaluate(X_test, y_test)[0]

138/138 [==============================] - 0s 4ms/step - loss: 0.2575 - accuracy: 0.9587


0.2575276494026184

In [0]:
def get_model(params):
    return Sequential([
        # Podwajamy liczbę filtrów                                      
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']), # powoduje że model się nie przyucza

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(params['dropout_cnn_block_two']),    

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(params['dropout_cnn_block_three']),            

        Flatten(),

        Dense(1024, activation = 'relu'), 
        Dropout(params['dropout_cnn_block_four']),

        Dense(1024, activation = 'relu'), 
        Dropout(params['dropout_cnn_block_five']),

        Dense(num_classes, activation = 'softmax'),         
    ])

In [0]:
# Przejmowanie parametrów

def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0,      
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [30]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_cnn_block_four': hp.uniform('dropout_cnn_block_four', 0.3, 0.7),
    'dropout_cnn_block_five': hp.uniform('dropout_cnn_block_five', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25, # Ilość iteracji
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_five': 0.4127802222109502, 'dropout_cnn_block_four': 0.541284370622112, 'dropout_cnn_block_one': 0.3874707834496014, 'dropout_cnn_block_three': 0.499309609335972, 'dropout_cnn_block_two': 0.43757663665989355}
accuracy=0.9510204195976257
{'batch_size': 180.0, 'dropout_cnn_block_five': 0.5031665783917498, 'dropout_cnn_block_four': 0.628746354270574, 'dropout_cnn_block_one': 0.5500760103895184, 'dropout_cnn_block_three': 0.3422193254830394, 'dropout_cnn_block_two': 0.4093433100299225}
accuracy=0.8514739274978638
{'batch_size': 190.0, 'dropout_cnn_block_five': 0.4727336485218948, 'dropout_cnn_block_four': 0.4191828530435826, 'dropout_cnn_block_one': 0.4905554594742041, 'dropout_cnn_block_three': 0.3400640278374624, 'dropout_cnn_block_two': 0.3207831306976203}
accuracy=0.8873015642166138
{'batch_size': 130.0, 'dropout_cnn_block_five': 0.4785704991842679, 'dropout_cnn_block_four': 0.5283560931565373, 'dropout_cnn_block_one': 0.3629129505470078, 'dropo

In [0]:
def get_model2(params):
    return Sequential([
        # Podwajamy liczbę filtrów                                      
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']), # powoduje że model się nie przyucza

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(params['dropout_cnn_block_two']),    

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),  
        Dropout(params['dropout_cnn_block_three']),            

        Flatten(),

        Dense(1024, activation = 'relu'), 
        Dropout(params['dropout_cnn_block_four']),

        Dense(1024, activation = 'relu'), 
        Dropout(params['dropout_cnn_block_five']),

        Dense(num_classes, activation = 'softmax'),         
    ])

In [0]:
def func_obj2(params):
  model = get_model2(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0,      
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [34]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_cnn_block_four': hp.uniform('dropout_cnn_block_four', 0.3, 0.7),
    'dropout_cnn_block_five': hp.uniform('dropout_cnn_block_five', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30, # Ilość iteracji
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_five': 0.390259189331945, 'dropout_cnn_block_four': 0.6213326003243385, 'dropout_cnn_block_one': 0.4073392859176802, 'dropout_cnn_block_three': 0.42972566461857337, 'dropout_cnn_block_two': 0.49393852497828744}
accuracy=0.9507936239242554
{'batch_size': 190.0, 'dropout_cnn_block_five': 0.32473480784069386, 'dropout_cnn_block_four': 0.36326641986023206, 'dropout_cnn_block_one': 0.4172728010197827, 'dropout_cnn_block_three': 0.40925071946713426, 'dropout_cnn_block_two': 0.44256890311382124}
accuracy=0.9630385637283325
{'batch_size': 130.0, 'dropout_cnn_block_five': 0.5535956454482072, 'dropout_cnn_block_four': 0.5366145082303109, 'dropout_cnn_block_one': 0.32116202150660594, 'dropout_cnn_block_three': 0.4331843891378503, 'dropout_cnn_block_two': 0.3956308432226063}
accuracy=0.9594104290008545
{'batch_size': 130.0, 'dropout_cnn_block_five': 0.4815739043066789, 'dropout_cnn_block_four': 0.3279106424049443, 'dropout_cnn_block_one': 0.3291487927368283